In [1]:
import pandas as pd
from faker import Faker
import random
import numpy as np

# Initialize Faker
fake = Faker()

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Define ranges for the number of records
num_medical_groups = random.randint(50, 100)
num_users = random.randint(500, 1000)
num_patients = random.randint(1000, 2000)
num_access_logs = random.randint(10000, 20000)

# Generate mock data for Medical_Group_List
medical_group_list = pd.DataFrame(
    {
        "medical_group_id": [fake.uuid4() for _ in range(num_medical_groups)],
        "groupname": [fake.company() for _ in range(num_medical_groups)],
        "state": [fake.state() for _ in range(num_medical_groups)],
        "date_group_created": [fake.date_time_this_decade() for _ in range(num_medical_groups)],
        "number_of_users": [random.randint(5, 20) for _ in range(num_medical_groups)],
        "number_of_patients": [random.randint(50, 200) for _ in range(num_medical_groups)],
    }
)

# Generate unique usernames
usernames = set()
while len(usernames) < num_users:
    usernames.add(fake.user_name())

# Generate mock data for Medical_Group_User
medical_group_user = pd.DataFrame(
    {
        "user_id": [fake.uuid4() for _ in range(num_users)],
        "medical_group_id": [random.choice(medical_group_list["medical_group_id"]) for _ in range(num_users)],
        "username": list(usernames),
        "password": [fake.password() for _ in range(num_users)],
        "role": [random.choice(["admin", "physician", "staff"]) for _ in range(num_users)],
        "date_user_created": [fake.date_time_this_decade() for _ in range(num_users)],
        "last_login_date": [fake.date_time_this_year() for _ in range(num_users)],
    }
)

# Generate mock data for Patient_Demographics
patient_demographics = pd.DataFrame(
    {
        "patient_id": [fake.uuid4() for _ in range(num_patients)],
        "age": [random.randint(0, 100) for _ in range(num_patients)],
        "weight": [round(random.uniform(50, 100), 1) for _ in range(num_patients)],  # weight in kg
        "last_visit_date": [fake.date_time_this_year() for _ in range(num_patients)],
    }
)

# Generate mock data for Patient_Access_Log
patient_access_log = pd.DataFrame(
    {
        "access_id": [fake.uuid4() for _ in range(num_access_logs)],
        "patient_id": [random.choice(patient_demographics["patient_id"]) for _ in range(num_access_logs)],
        "user_id": [random.choice(medical_group_user["user_id"]) for _ in range(num_access_logs)],
        "date_accessed": [fake.date_time_this_year() for _ in range(num_access_logs)],
    }
)

# Display Head of each base dataframe
for dataframe in [medical_group_list, medical_group_user, patient_demographics, patient_access_log]:
    print(dataframe.head())

                       medical_group_id                   groupname  \
0  4a9155c2-1477-44f1-a48d-c9ef9e16e1ac                  Sexton LLC   
1  edd8af25-d77e-41cc-b15b-0b6a744fdaa8   Martin, Miller and Medina   
2  bcb66edf-6148-4ccd-8b20-a64658706579            Johnson-Crawford   
3  d968e5a9-3f55-4c6c-995c-8506ae89c687                  Thomas PLC   
4  a79c646a-b4d2-4e57-b064-e0745087d1c6  Nguyen, Gregory and George   

        state         date_group_created  number_of_users  number_of_patients  
0    New York 2021-01-12 11:33:55.404722               12                  62  
1  New Mexico 2023-05-10 20:59:11.135803               12                  78  
2  New Jersey 2021-12-24 23:14:42.878878                9                  89  
3  California 2022-06-04 15:55:43.601038                8                  90  
4    Delaware 2021-05-04 21:54:51.531410                7                 158  
                                user_id                      medical_group_id  \
0  2ddf33f6-

In [2]:
import duckdb

# Create a DuckDB file
con = duckdb.connect(database="stellar_health.db")

# Create schema 'stellar_health'
con.execute("CREATE SCHEMA IF NOT EXISTS stellar_health_application")

# Load data into DuckDB within the 'stellar_health' schema
con.execute(
    "CREATE OR REPLACE TABLE stellar_health.stellar_health_application.medical_group_list AS SELECT * FROM medical_group_list"
)
con.execute(
    "CREATE OR REPLACE TABLE stellar_health.stellar_health_application.medical_group_user AS SELECT * FROM medical_group_user"
)
con.execute(
    "CREATE OR REPLACE TABLE stellar_health.stellar_health_application.patient_demographics AS SELECT * FROM patient_demographics"
)
con.execute(
    "CREATE OR REPLACE TABLE stellar_health.stellar_health_application.patient_access_log AS SELECT * FROM patient_access_log"
)

# Verify the data is loaded
print(con.execute("SELECT * FROM stellar_health.stellar_health_application.medical_group_list LIMIT 5").fetchdf())
print(con.execute("SELECT * FROM stellar_health.stellar_health_application.medical_group_user LIMIT 5").fetchdf())
print(con.execute("SELECT * FROM stellar_health.stellar_health_application.patient_demographics LIMIT 5").fetchdf())
print(con.execute("SELECT * FROM stellar_health.stellar_health_application.patient_access_log LIMIT 5").fetchdf())

                       medical_group_id                   groupname  \
0  4a9155c2-1477-44f1-a48d-c9ef9e16e1ac                  Sexton LLC   
1  edd8af25-d77e-41cc-b15b-0b6a744fdaa8   Martin, Miller and Medina   
2  bcb66edf-6148-4ccd-8b20-a64658706579            Johnson-Crawford   
3  d968e5a9-3f55-4c6c-995c-8506ae89c687                  Thomas PLC   
4  a79c646a-b4d2-4e57-b064-e0745087d1c6  Nguyen, Gregory and George   

        state         date_group_created  number_of_users  number_of_patients  
0    New York 2021-01-12 11:33:55.404722               12                  62  
1  New Mexico 2023-05-10 20:59:11.135803               12                  78  
2  New Jersey 2021-12-24 23:14:42.878878                9                  89  
3  California 2022-06-04 15:55:43.601038                8                  90  
4    Delaware 2021-05-04 21:54:51.531410                7                 158  
                                user_id                      medical_group_id  \
0  2ddf33f6-

# Scenario 1 Questions
## 1. Describe your high level approach to solving this problem, assuming all the tech stack is already up and operational.

## 2. Design an ER diagram based on the App Tables listed above that you intend to implement in dbt and share your work.
## 3. Write SQL statements based on your new dbt model that answer the following questions:
### a. How many patient records have been accessed by 5 or more different users in the past 14 months?
### b. Which medical groups are focused on treating patients who are retired?
## 4. Are there dbt features that would make solving for this problem straightforward?
## 5. How would you communicate/train your team member Data Analysts on how to best use your new dbt model(s)?

In [3]:
print(con.execute("SELECT * FROM main.raw_stellar_health_application__medical_group_list  LIMIT 5").fetchdf())

CatalogException: Catalog Error: Table with name raw_stellar_health_application__medical_group_list does not exist!
Did you mean "stellar_health_application.medical_group_list"?
LINE 1: SELECT * FROM main.raw_stellar_health_application__me...
                      ^

In [4]:
# Close the connection
con.close()